In [1]:
import os, glob
import json

import numpy as np
import pandas as pd


In [43]:
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

lookup["urn:cts:greekLit:tlg0640.tlg001"]

['tlg0640.tlg001.ogl-eng1.xml', 'alciphron_1896']

In [44]:
for path in ["/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1.xml"]:
    prefix = os.path.splitext(path)[0]
    print(prefix)

/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1


In [45]:
text_ctsurn = []
num_transl = []
transl_langs = []
num_src_sents = []
text_top_1s = []
text_top_5s = []
text_top_10s = []
num_tgt_sents = []
num_nulls_all_transl = []
names_translations = []
transl_top10s = []

not_transl_keys = ["num_src_sents", "num_translations", "text_top1", "text_top5", "text_top10"]

eval_dicts_dir = "/home/craig.car/repos/chiron/chironata/eval_datafiles/"
for eval_json_path in glob.iglob(eval_dicts_dir+"*.json"):
    ctsurn = os.path.splitext(os.path.basename(eval_json_path))[0]
    with open(eval_json_path) as f:
        eval_json = json.load(f)
    text_ctsurn.append(ctsurn)
    # print(eval_json["apolloniusRhodius_1_1791"].keys())
    num_transl.append(eval_json["num_translations"])
    num_src_sents.append(eval_json["num_src_sents"])
    text_top_1s.append(eval_json["text_top1"])
    text_top_5s.append(eval_json["text_top5"])
    text_top_10s.append(eval_json["text_top10"])
    text_langs = set()
    num_transl_sents = 0
    num_nulls_all_tranls = 0
    transl_names = []
    top10s_transl = []
    for key in eval_json.keys():
        if key not in not_transl_keys:
            # print(eval_json[key].keys())
            text_langs.add(eval_json[key]["tgt_lang"])
            num_transl_sents += eval_json[key]["num_tgt_sents"]
            num_nulls_all_tranls += eval_json[key]["num_nulls"]
            transl_names.append(key)
            top10s_transl.append("%.3f" % eval_json[key]["transl_top10"])
            # print(num_transl_sents)
    transl_langs.append(text_langs)
    num_tgt_sents.append(num_transl_sents)
    num_nulls_all_transl.append(num_nulls_all_tranls)
    names_translations.append(transl_names)
    transl_top10s.append(top10s_transl)


In [46]:
print(len(text_ctsurn))
print(len(num_transl))
print(len(transl_langs))
print(len(num_src_sents))
print(len(text_top_1s))
print(len(text_top_5s))
print(len(text_top_10s))
print(len(num_tgt_sents))
print(len(num_nulls_all_transl))
print(len(names_translations))
print(len(transl_top10s))

709
709
709
709
709
709
709
709
709
709
709


In [47]:
eval_df = pd.DataFrame(text_ctsurn)
eval_df.columns = ["ctsurn"]

In [48]:
eval_df["number_src_sents"] = num_src_sents
eval_df["number_translations"] = num_transl
eval_df["names_translations"] = names_translations
eval_df["translation_languages"] = transl_langs
eval_df["num_tgt_sents_all_translations"] = num_tgt_sents
eval_df["num_src2null_all_translations"] = num_nulls_all_transl
eval_df["text_wide_recall@1"] = text_top_1s
eval_df["text_wide_recall@5"] = text_top_5s
eval_df["text_wide_recall@10"] = text_top_10s
eval_df["translations_recall@10"] = transl_top10s

In [49]:
eval_df.round(3)

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
0,urn:cts:greekLit:tlg0001.tlg001,1738,3,"[apolloniusRhodius_1_1791, tlg0001.tlg001.opp-...","{it, fr, en}",18327,0,0.179,0.296,0.353,"[0.024, 0.605, 0.432]"
1,urn:cts:greekLit:tlg0003.tlg001,4419,7,"[thucydides_1_1864, thucydides_1_1869, thucydi...","{de, fr, en}",53054,547,0.130,0.194,0.225,"[0.698, 0.659, 0.010, 0.156, 0.010, 0.010, 0.032]"
2,urn:cts:greekLit:tlg0004.tlg001,6440,1,[diogenesLaertius_1853],{en},11907,3,0.436,0.654,0.737,[0.737]
3,urn:cts:greekLit:tlg0005.tlg001,549,5,"[theocritus_1855, theocritus_1878, theocritus_...","{de, fr, en}",23922,1,0.234,0.409,0.499,"[0.468, 0.605, 0.576, 0.470, 0.377]"
4,urn:cts:greekLit:tlg0005.tlg002,94,5,"[theocritus_1855, theocritus_1878, theocritus_...","{de, fr, en}",23922,0,0.123,0.200,0.262,"[0.298, 0.447, 0.181, 0.234, 0.149]"
...,...,...,...,...,...,...,...,...,...,...,...
704,urn:cts:latinLit:stoa0255.stoa010,1539,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.664,0.858,0.919,"[0.905, 0.933]"
705,urn:cts:latinLit:stoa0255.stoa011,110,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.686,0.855,0.914,"[0.855, 0.973]"
706,urn:cts:latinLit:stoa0255.stoa012,295,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.646,0.814,0.863,"[0.769, 0.956]"
707,urn:cts:latinLit:stoa0255.stoa013,460,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.687,0.872,0.936,"[0.915, 0.957]"


In [50]:
# tlg0237.tlg002.ogl-eng1.xml

In [55]:
eval_df.loc[eval_df["ctsurn"]=="urn:cts:greekLit:tlg0019.tlg004", "names_translations"].tolist()

[['tlg0019.tlg004.ogl-eng2',
  'aristophanes_1_1858',
  'aristophanes_1873',
  'aristophanes_1_1915',
  'aristophanes_1875',
  'aristophanes_2_1915',
  'aristophanes_3_1830',
  'aristophanes_1_1830',
  'aristophanes_1881_1',
  'aristophanes_1907_1']]

In [56]:
eval_df.loc[eval_df["ctsurn"]=="urn:cts:greekLit:tlg0019.tlg004", "translations_recall@10"].tolist()

[['0.437',
  '0.579',
  '0.309',
  '0.046',
  '0.437',
  '0.219',
  '0.282',
  '0.058',
  '0.127',
  '0.046']]

In [57]:
eval_df.loc[eval_df["ctsurn"]=="urn:cts:greekLit:tlg0019.tlg004", "number_translations"].tolist()

[10]

In [12]:
eval_df.sort_values("text_wide_recall@10", ascending=[False])

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
661,urn:cts:latinLit:phi1212.phi003,392,1,[apuleius_1878],{en},8847,0,0.757653,0.989796,1.043367,[1.043]
595,urn:cts:latinLit:phi0474.phi055,2011,1,[cicero_1874],{en},5692,0,0.776231,0.945301,0.987568,[0.988]
666,urn:cts:latinLit:phi1348.abo011,517,2,"[suetonius_1855-1897, caesar_commentaries_1867]","{de, fr}",13468,0,0.709865,0.913926,0.984526,"[0.971, 0.998]"
433,urn:cts:greekLit:tlg0074.tlg004,282,1,[arrian_1860],{fr},3216,0,0.549645,0.872340,0.971631,[0.972]
323,urn:cts:greekLit:tlg0034.tlg001,406,1,[lycurgus_1869],{de},655,0,0.655172,0.874384,0.963054,[0.963]
...,...,...,...,...,...,...,...,...,...,...,...
597,urn:cts:latinLit:phi0474.phi057,10677,3,"[cicero_1874, cicero_1855-1897_2, cicero_1855-...","{de, en}",17058,7364,0.002716,0.007586,0.012394,"[0.010, 0.017, 0.011]"
463,urn:cts:greekLit:tlg0096.tlg002,6062,1,[aesopHesiodQuintus_1865-1866],{de},7368,1,0.000660,0.006434,0.012042,[0.012]
596,urn:cts:latinLit:phi0474.phi056,6913,2,"[cicero_1855-1897_2, cicero_1855-1903_1]",{de},11366,1032,0.001374,0.004484,0.008969,"[0.010, 0.008]"
582,urn:cts:latinLit:phi0474.phi035,4047,2,"[cicero_1855-1892_11, cicero_1868_12]",{de},9216,10,0.000741,0.005313,0.008772,"[0.010, 0.008]"


In [13]:
eval_df.sort_values("text_wide_recall@10", ascending=[False]).head(25)

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
661,urn:cts:latinLit:phi1212.phi003,392,1,[apuleius_1878],{en},8847,0,0.757653,0.989796,1.043367,[1.043]
595,urn:cts:latinLit:phi0474.phi055,2011,1,[cicero_1874],{en},5692,0,0.776231,0.945301,0.987568,[0.988]
666,urn:cts:latinLit:phi1348.abo011,517,2,"[suetonius_1855-1897, caesar_commentaries_1867]","{de, fr}",13468,0,0.709865,0.913926,0.984526,"[0.971, 0.998]"
433,urn:cts:greekLit:tlg0074.tlg004,282,1,[arrian_1860],{fr},3216,0,0.549645,0.872340,0.971631,[0.972]
323,urn:cts:greekLit:tlg0034.tlg001,406,1,[lycurgus_1869],{de},655,0,0.655172,0.874384,0.963054,[0.963]
659,urn:cts:latinLit:phi1212.phi001,1314,1,[apuleius_1878],{en},8847,0,0.705479,0.898021,0.953577,[0.954]
615,urn:cts:latinLit:phi0588.abo015,89,2,"[justin_cNepos_eutropius_1902, corneliusNepos_...","{de, en}",12161,0,0.735955,0.870787,0.938202,"[0.933, 0.944]"
707,urn:cts:latinLit:stoa0255.stoa013,460,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.686957,0.871739,0.935870,"[0.915, 0.957]"
708,urn:cts:latinLit:stoa0255.stoa014,435,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.673563,0.859770,0.931034,"[0.961, 0.901]"
699,urn:cts:latinLit:stoa0255.stoa004,387,2,"[seneca_1877, seneca_1889]","{fr, en}",36068,0,0.670543,0.863049,0.928941,"[0.904, 0.953]"


In [14]:
eval_df["number_src_sents"].sum()

579711

In [15]:
eval_df["num_tgt_sents_all_translations"].sum()

16460463

In [16]:
eval_df["number_translations"].sum()

2108

In [17]:
eval_df.sort_values("num_src2null_all_translations")

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
0,urn:cts:greekLit:tlg0001.tlg001,1738,3,"[apolloniusRhodius_1_1791, tlg0001.tlg001.opp-...","{it, fr, en}",18327,0,0.178941,0.296318,0.353471,"[0.024, 0.605, 0.432]"
426,urn:cts:greekLit:tlg0062.tlg065,204,1,[lucian_1829_10],{de},4222,0,0.348039,0.544118,0.696078,[0.696]
428,urn:cts:greekLit:tlg0062.tlg067,213,4,"[lucian_1888, lucian_1867_3, lucian_1866_1, lu...","{de, fr, en}",29213,0,0.322770,0.468310,0.534038,"[0.859, 0.141, 0.728, 0.408]"
429,urn:cts:greekLit:tlg0062.tlg068,425,4,"[lucian_1888, lucian_1867_3, lucian_1866_1, lu...","{de, fr, en}",29213,0,0.327059,0.450000,0.525882,"[0.802, 0.104, 0.727, 0.471]"
430,urn:cts:greekLit:tlg0062.tlg069,507,3,"[lucian_1831_13, Lucian_1827_ht8, lucian_1827_7]",{de},9872,0,0.097962,0.174227,0.226167,"[0.540, 0.059, 0.079]"
...,...,...,...,...,...,...,...,...,...,...,...
465,urn:cts:greekLit:tlg0525.tlg001,7163,7,"[pausanias_2_nd, pausanias_2_1817, pausanias_1...","{de, it, fr, en}",33812,11040,0.056401,0.076783,0.088770,"[0.021, 0.021, 0.227, 0.010, 0.013, 0.019, 0.310]"
531,urn:cts:greekLit:tlg2000.tlg001,12496,3,"[plotinus_2_1905, plotinus_1895, plotinus_1_1905]","{de, en}",14361,17386,0.029583,0.049536,0.058552,"[0.073, 0.060, 0.043]"
255,urn:cts:greekLit:tlg0016.tlg001,6998,13,"[herodotus_2_1850, herodotus_1_1802, herodotus...","{it, fr, en}",73657,19690,0.107009,0.152186,0.173137,"[0.003, 0.115, 0.777, 0.012, 0.130, 0.023, 0.3..."
637,urn:cts:latinLit:phi0914.phi001,30809,4,"[livius_1853_1, livy_2_1860, livy_1_1860, livi...","{de, fr}",38508,70085,0.085965,0.102738,0.109489,"[0.252, 0.013, 0.039, 0.134]"
